In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sktime.forecasting.model_selection import SlidingWindowSplitter
from sklearn.model_selection import TimeSeriesSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
import matplotlib.pyplot as plt

In [34]:
import multiprocessing

In [44]:
rs=121
ns=5
stocks=['aapl','googl','nvda','dal','xom','cvx','vz']
testing_size=500
tss=TimeSeriesSplit(n_splits=ns,test_size=testing_size)
spread_range = range(2, 25)

In [17]:
init_dir_path = str(os.getcwd())
data=pd.read_csv(init_dir_path+'/stock_hmm_output.csv',engine='python').drop(['Unnamed: 0'],axis=1)
features=['Return','Volatility','HighR','LowR']
target='State'

In [18]:
#With TimeSeries
#First Set Aside Testing data
final_test=[]
l=[]
ft_size=400
for x in data.groupby(['Ticker']):
    final_test.append(x[1][-ft_size:])
    l.append(x[1][:-ft_size])

In [19]:
def createSlidingWindow(tr,te,f,t, window=7):
    splitting=SlidingWindowSplitter(window_length=window,step_length=1,fh=[1])
    train_x=[]
    train_y=[]
    test_x=[]
    test_y=[]
    
    for i in splitting.split(tr):
        train_x.append(tr.iloc[i[0]][f].values.flatten())
        train_y.append(tr.iloc[i[1]][t].values.flatten()[0])
    
    for i in splitting.split(te):
        test_x.append(te.iloc[i[0]][f].values.flatten())
        test_y.append(te.iloc[i[1]][t].values.flatten()[0])
    return ((train_x,train_y),(test_x,test_y))

In [15]:
testing_size=500

tss=TimeSeriesSplit(n_splits=ns,test_size=testing_size)
training_list=[]
testing_list=[]
for i in range(ns):
    training_list.append([[],[]])
    testing_list.append([[],[]])
training_lists = {}
testing_lists = {}
for spread in range(7,21):
    for x in l:
        for (i,t) in enumerate(tss.split(x)):
            ((train_x,train_y),(test_x,test_y))=createSlidingWindow(x.iloc[t[0]],x.iloc[t[1]],features,target, window = spread)
            training_list[i][0] += train_x
            training_list[i][1] += train_y
            testing_list[i][0] += test_x
            testing_list[i][1] += test_y
    training_lists[spread] = training_list
    testing_lists[spread] = testing_list
    training_list=[]
    testing_list=[]
    for i in range(ns):
        training_list.append([[],[]])
        testing_list.append([[],[]])

In [16]:
with open('saved_training_lists.pkl', 'wb') as f:
    pickle.dump(training_lists, f)

with open('saved_testing_lists.pkl', 'wb') as f:
    pickle.dump(testing_lists, f)


In [21]:
with open('saved_training_lists.pkl', 'rb') as f:
    training_lists = pickle.load(f)

with open('saved_testing_lists.pkl', 'rb') as f:
    testing_lists = pickle.load(f)

In [26]:
def accuracies_model(model_dict, window, stock_ind, map=True):
    a=0
    conf_mat=np.zeros((2,2))
    if map:
        model = model_dict[window][stock_ind]
    else:
        model = model_dict
    for i in range(ns):
        model.fit(X=training_lists[window][stock_ind][i][0],y=training_lists[window][stock_ind][i][1])
        y_pred=model.predict(testing_lists[window][stock_ind][i][0])
        y_true=testing_lists[window][stock_ind][i][1]
        a+= accuracy_score(y_true,y_pred)
        conf_mat=conf_mat+confusion_matrix(y_true=y_true,y_pred=y_pred)
    a=a/ns
    return (a,conf_mat)

In [23]:
def plot_accuracies(accuracies, model_type, acc_type="accuracy"): #accuracies needs to be a nested dictionary window->stock index
    for stock_ind in range(len(stocks)):
        vals = []
        for window in accuracies.keys():
            vals.append(accuracies[window][stock_ind])
        plt.figure(figsize=(6, 4))
        plt.plot(list(accuracies.keys()), vals, marker='o')
        plt.title(f"{model_type} accuracy scores for {stocks[stock_ind]}")
        plt.xlabel("Window size")
        plt.ylabel(acc_type)
        plt.tight_layout()
        plt.show()

In [ ]:
#Find best models
window=7
model=AdaBoostClassifier()
tree_depths=[DecisionTreeClassifier(max_depth=x) for x in range(6)]

parameters = {
    'n_estimators' : range(100,250,20),
    'learning_rate' : [0.5,0.7,0.9,1,1.1,1.3],
    'estimator' : tree_depths
}
grid_searches = [GridSearchCV(model, parameters, cv = 5, n_jobs=-1) for i in range(len(stocks))]
results=[]
for stock_ind in range(len(stocks)):
    grid_searches[stock_ind].fit(X=training_lists[window][stock_ind][-1][0],y=training_lists[window][stock_ind][-1][1])

In [ ]:
models_best=[x.best_estimator_ for x in grid_searches]

In [ ]:
accuracies_ada = {}
confusion_mat = {}
f1_score={}
#ada_model = AdaBoostClassifier(n_estimators=100,random_state=121)
for window in spread_range:
    accuracies_ada[window] = {}
    confusion_mat[window] = {}
    f1_score[window]={}
    def helper(i):
        return accuracies_model(models_best[i],window,i,map=False)
    pool=multiprocessing.Pool(len(stocks))
    results=list(pool.map(helper, range(len(stocks))))
    for stock_ind in range(len(stocks)):
        accuracies_ada[window][stock_ind] = results[stock_ind][0]
        confusion_mat[window][stock_ind] = results[stock_ind][1]
        A=confusion_mat[window][stock_ind]
        f1_score[window][stock_ind]=(2*A[0][0]/(2*A[0][0]+A[0][1]+A[1][0]))
        


In [1]:
plot_accuracies(f1_score, "Ada Boost",acc_type="f1_score")
plot_accuracies(accuracies_ada, "Ada Boost")

NameError: name 'plot_accuracies' is not defined

In [ ]:
#13 seems best from above

    #results.append(grid_search)

#a=[] 
#for i in range(ns):
#    grid_search.fit(X=training_list[i][0],y=training_list[i][1])
#    print(grid_search.best_estimator_)
#    print(grid_search.best_score_)
#    a.append(grid_search.cv_results_)

    #a+= accuracy_score(model.predict(testing_list[i][0]),testing_list[i][1])
#a=a/ns

/home/srijan/miniconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
240 fits failed out of a total of 1440.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "/home/srijan/miniconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/srijan/miniconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/sr